In [1]:
import pandas as pd
import numpy as np
import time
from collections import defaultdict
import datetime
# from datetime import datetime
from scipy import stats
# from datetime import timedelta
import matplotlib.pyplot as plt
import math
import os
import gc

In [144]:
# 資料儲存路徑&參數設置
initial_place = "D:/user/文件/python/Eric/"
# 測試時需要確認此日期在資料夾內 
Start_date = "20110614"
start_file = 'FTPOSF_'+Start_date+'.csv'
End_date = "20110614"
end_file = 'FTPOSF_'+End_date+'.csv'

poi_file_start = 'FTPPOI_'+ Start_date[:6]+'.t.csv'
poi_file_end = 'FTPPOI_'+ End_date[:6]+'.t.csv'

name = ["OSF","MTF","DSP","POI"]
poi_count = 0
poi_true = True
year_month_now = ""
date = ""
date_now = ""
Folder_Path_ini = initial_place + "\\" + "OSF"
os.chdir(Folder_Path_ini)
file_list = np.array(os.listdir())

In [39]:
#挑出TXF和單式委託
def Data_Pick():
    OSF_TXF = OSF_RawData[(OSF_RawData['merchandise'] == 'TXF')&(OSF_RawData['sc']=='S')].reset_index(drop=True)
    MTF_TXF = MTF_RawData[(MTF_RawData['merchandise'] == 'TXF')&(MTF_RawData['sc']=='S')].reset_index(drop=True)
    DSP_TXF = DSP_RawData[(DSP_RawData['merchandise'] == 'TXF')&(DSP_RawData['sc']=='S')].reset_index(drop=True)
    POI_TXF = POI_RawData[(POI_RawData['merchandise'] == 'TXF')].reset_index(drop=True)     

    OSF_TXF = OSF_TXF.sort_values(by=['daytime']).reset_index(drop=True)
    #columns 預處理
    OSF_TXF['date'] = OSF_TXF['daytime'].dt.date
    OSF_TXF['time'] = OSF_TXF['daytime'].dt.time

    MTF_TXF['date'] = MTF_TXF['daytime'].dt.date

    POI_TXF['TotalVol'] = POI_TXF['tbuyv'] + POI_TXF['tsellv']
    POI_TXF['TotalPOI'] = POI_TXF['tbuyre'] + POI_TXF['tsellre']
    POI_TXF['date'] = POI_TXF['daytime'].dt.date

    DSP_TXF['date'] = DSP_TXF['daytime'].dt.date
    return OSF_TXF, MTF_TXF, DSP_TXF, POI_TXF

In [40]:
# %%time
#每月判斷一次轉倉日期避掉迴圈
def Transfer():
    Transdaylist1 = [20070118, 20070227, 20070322, 20070419, 20070517, 20070621, 20070719, 20070816, 20070920, 20071018,
                   20071122, 20071220, 20080117, 20080221, 20080320, 20080417, 20080522, 20080619, 20080717, 20080821,
                   20080918, 20081016, 20081120, 20081217, 20090121, 20090218, 20090318, 20090415, 20090520, 20090617,
                   20090715, 20090819, 20090916, 20091021, 20091118, 20091216, 20100120, 20100222, 20100317, 20100421,
                   20100519, 20100617, 20100721, 20100818, 20100915, 20101020, 20101117, 20101215, 20110119, 20110216,
                   20110316, 20110420, 20110518, 20110615, 20110720, 20110817, 20110921, 20111019, 20111116, 20111221]
    labels = ['200702', '200703', '200704', '200705', '200706', '200707', '200708', '200709', '200710',
              '200711', '200712', '200801', '200802', '200803', '200804', '200805', '200806', '200807', '200808',
              '200809', '200810', '200811', '200812', '200901', '200902', '200903', '200904', '200905', '200906',
              '200907', '200908', '200909', '200910', '200911', '200912', '201001', '201002', '201003', '201004',
              '201005', '201006', '201007', '201008', '201009', '201010', '201011', '201012', '201101', '201102',
              '201103', '201104', '201105', '201106', '201107', '201108', '201109', '201110', '201111', '201112']
    OSF_TXF1 = OSF_TXF
    OSF_TXF1['date_cut'] = OSF_TXF1['date'].apply(lambda x : int(x.strftime('%Y%m%d')))
    OSF_TXF1['labels'] = pd.cut(OSF_TXF1['date_cut'], Transdaylist1, right = False, labels = labels)
    OSF_TXF1['contract1'] = OSF_TXF1['contract1'].apply(lambda x: str(x))
    OSF_target = OSF_TXF1[OSF_TXF1['contract1']==OSF_TXF1['labels']]
    OSF_target = OSF_target.drop(['labels','date_cut'],axis= 1).reset_index(drop=True)

    POI_TXF1 = POI_TXF
    POI_TXF1['date_cut'] = POI_TXF1['date'].apply(lambda x : int(x.strftime('%Y%m%d')))
    POI_TXF1['labels'] = pd.cut(POI_TXF1['date_cut'], Transdaylist1, right = False, labels = labels)
    POI_TXF1['contract'] = POI_TXF1['contract'].apply(lambda x: str(x))
    POI_target = POI_TXF1[(POI_TXF1['contract']==POI_TXF1['labels'])&(POI_TXF1['TotalVol']>0)]
    POI_target = POI_target.drop(['labels','date_cut'],axis= 1).reset_index(drop=True)

    MTF_TXF1 = MTF_TXF
    MTF_TXF1['date_cut'] = MTF_TXF1['date'].apply(lambda x : int(x.strftime('%Y%m%d')))
    MTF_TXF1['labels'] = pd.cut(MTF_TXF1['date_cut'], Transdaylist1, right = False, labels = labels)
    MTF_TXF1['contract1'] = MTF_TXF1['contract1'].apply(lambda x: str(x))
    MTF_target = MTF_TXF1[MTF_TXF1['contract1']==MTF_TXF1['labels']]
    MTF_target = MTF_target.drop(['labels','date_cut'],axis= 1).reset_index(drop=True)

    DSP_TXF1 = DSP_TXF
    DSP_TXF1['date_cut'] = DSP_TXF1['date'].apply(lambda x : int(x.strftime('%Y%m%d')))
    DSP_TXF1['labels'] = pd.cut(DSP_TXF1['date_cut'], Transdaylist1, right = False, labels = labels)
    DSP_TXF1['contract1'] = DSP_TXF1['contract1'].apply(lambda x: str(x))
    DSP_target = DSP_TXF1[DSP_TXF1['contract1']==DSP_TXF1['labels']]
    DSP_target = DSP_target.drop(['labels','date_cut'],axis= 1).reset_index(drop=True)
    return OSF_target, POI_target, MTF_target, DSP_target

In [58]:
# %%time
#運算並記錄成交量/委託量/執行率/持倉量/持倉時間
def Vol_Calculate():
    #運算並記錄成交量/委託量/執行率/持倉量/持倉時間
    EexeVol = MTF_target.groupby(['date','id'])['vol1'].sum()       
    EexeVol = EexeVol.reset_index()                                 #成交從MTF檔中加總計算

    OrderVol = OSF_target.groupby(['date','id'])['vol'].sum() 
    OrderVol = OrderVol.reset_index()                               #委託數量從OSF檔中計算有效委託數量

    Record_df = pd.merge(EexeVol,OrderVol, on=['date','id'], how='outer').rename(columns={"vol1": "ExeVol", 'vol': "OrderVol"}).fillna(0)
    Record_df['ExeRate'] = Record_df['ExeVol']/Record_df['OrderVol']  #將成交及有效委託數量merge之後再計算委託/成交比

    #將存貨檔與委託檔合併挑出日內有委託or有成交or有部位的交易人
    #這個資料即會將持有部位但是未委託和成交的交易人也納入須要進一步討論
#     HoldingPeriod_temp = HoldingPeriod[(HoldingPeriod['date']==datetime.date(2011,6,14))|(HoldingPeriod['date']==datetime.date(2011,6,15))|(HoldingPeriod['date']==datetime.date(2011,6,16))]   #平均持倉時間是比較健康的篩選條件
#     HoldingPeriod_temp = HoldingPeriod_temp[['date','id','HoldingPeriod']]

    Record_df = pd.merge(Record_df,HoldingPeriod, on=['date','id'], how = 'outer')  
    Record_df = Record_df.fillna(0)
    Record_df = Record_df[Record_df['HoldingPeriod'] > datetime.timedelta(0)]              #剔除有委託沒成交沒持倉的交易人

    #(高委託/成交比+高交易量+低庫存)挑出每日快速交易者
    Record_df1 = Record_df
    Datelist1 = Record_df1['date'].unique()
    RankData = pd.DataFrame(columns = Record_df1.columns)        #建一個儲存器
    for i in Datelist1:
        Datatoday = Record_df1[Record_df1['date']==i]
        Datatoday['ExeVolScore'] = Datatoday['ExeVol'].apply(lambda x: stats.percentileofscore(Datatoday['ExeVol'], x))
        Datatoday['OrderVolScore'] = Datatoday['OrderVol'].apply(lambda x: stats.percentileofscore(Datatoday['OrderVol'], x))
        Datatoday['ExeRateScore'] = Datatoday['ExeRate'].apply(lambda x: stats.percentileofscore(Datatoday['ExeRate'], x))
        Datatoday['HoldingPeriod'] = Datatoday['HoldingPeriod'].apply(lambda x:x.total_seconds())
        Datatoday['HoldingPeriodScore'] = Datatoday['HoldingPeriod'].apply(lambda x: stats.percentileofscore(Datatoday['HoldingPeriod'], x)) 
        RankData = RankData.append(Datatoday)
        Datatoday = []

    #記錄每日快速交易者
    Datelist = pd.unique(POI_target['date'])
    Threshold = 20
    RankData_HFT = RankData[(RankData['HoldingPeriodScore']<=Threshold)&(RankData['OrderVolScore']>=(100-Threshold))&
        (RankData['ExeVolScore']>=(100-Threshold))&(RankData['ExeRateScore']<=Threshold)]
    Record_HFT = RankData_HFT[['date','id']].reset_index(drop=True)
    return Record_HFT,RankData

In [59]:
#篩選出每日快速交易人
# Threshold = [0,10,20,30,40,50,60,70,80,90]
def Features_Record(name):
    #紀錄交易人特性基本統計量
    Threshold = [10,20,30,40,50,60,70,80,90]
    Features_record = pd.DataFrame(columns =['HoldingPeriod','OrderVol', 'ExeVol', 'ExeRate'],index=range(len(Threshold)))
    for i in range(len(Threshold)):
        condition1 = (RankData['HoldingPeriodScore']<= Threshold[i])
        condition2 = (RankData['OrderVolScore']>= Threshold[8-i])
        condition3 = (RankData['ExeVolScore']>= Threshold[8-i])
        condition4 = (RankData['ExeRateScore']<= Threshold[i])
        Features_record.loc[i,'HoldingPeriod'] = RankData[(condition1)]['HoldingPeriod'].mean()
        Features_record.loc[i,'OrderVol'] = RankData[(condition2)]['OrderVol'].mean()
        Features_record.loc[i,'ExeVol'] = RankData[(condition3)]['OrderVol'].mean()
        Features_record.loc[i,'ExeRate'] = RankData[(condition4)]['ExeRate'].mean()
        try:
            Features_record.loc[i,'Order/ExeRatio']= 1/Features_record.loc[i,'ExeRate']
        except:
            Features_record.loc[i,'Order/ExeRatio']=0
    Features_record.to_csv('Features_record'+ date +name+'.csv')
    return Features_record

In [128]:
# %%time
#切割MTF檔計算HFT參與率
# 切割所屬樣本
#先創一段時間樣本讓並記錄OSF委託所屬樣本區間
def JoinRate5M():
    Time_list = [845, 850, 855, 900, 905 ,910, 915, 920, 925, 930, 935, 940, 945, 950, 955, 1000, 1005, 1010, 1015, 1020, 1025, 1030, 1035,
                1040, 1045, 1050, 1055, 1100, 1105, 1110, 1115, 1120, 1125, 1130, 1135, 1140, 1145, 1150, 1155, 1200, 1205, 1210,1215,
                1220, 1225, 1230, 1235, 1240, 1245, 1250, 1255, 1300, 1305, 1310, 1315, 1320, 1325, 1330, 1335, 1340, 1346]
    Time_labels =['850', '855', "900", "905" ,"910", "915", "920", "925", "930", "935", "940", "945", "950", "955", "1000", "1005", "1010",
                  "1015", "1020", "1025", "1030", "1035", "1040", "1045", "1050", "1055", "1100", "1105", "1110", '1115', '1120', '1125', 
                  '1130', '1135', '1140', '1145', '1150', '1155', '1200', '1205', '1210', '1215', '1220', '1225', '1230', '1235', '1240',
                  '1245', '1250', '1255', '1300', '1305', '1310', '1315', '1320', '1325', '1330', '1335', '1340', '1346']
    MTF_target2 = MTF_target.copy()
    MTF_target2['time'] = MTF_target2['daytime'].apply(lambda x: (x.hour)*100+x.minute)
    MTF_target2['Time_labels'] = pd.cut(MTF_target2['time'], Time_list, right = False, labels = Time_labels)

    #貼標記給所有HFT後計算參與率
    MTF_target2['HFT'] = 0
    for i in range(len(Record_HFT)):
        MTF_target2.loc[(Record_HFT.loc[i,'date']==MTF_target2['date'])&(Record_HFT.loc[i,'id']==MTF_target2['id']),'HFT'] = 1
    MTF_target2['HFTVol'] = MTF_target2['HFT']*MTF_target2['vol1']
    HFT_JoinRate = (MTF_target2.groupby(['date','Time_labels'])['HFTVol'].sum()/MTF_target2.groupby(['date','Time_labels'])['vol1'].sum())
    HFT_JoinRate = HFT_JoinRate.reset_index().rename(columns={0:'HFT_JoinRate'})

    #計算HFT流動性供需
    MTF_targetSell = MTF_target2[MTF_target2['bs1']=='S']
    MTF_targetSell['UpDown'] = MTF_targetSell['price1']<MTF_targetSell['price1'].shift()
    MTF_targetSell['Fluidity'] = 0
    MTF_targetSell['HFT_SellDemand'] = MTF_targetSell['UpDown']*MTF_targetSell['HFT']*MTF_targetSell['vol1']
    HFT_SellDemand = (MTF_targetSell.groupby(['date','Time_labels'])['HFT_SellDemand'].sum()/MTF_targetSell['vol1'].sum()).reset_index()

    MTF_targetBuy = MTF_target2[MTF_target2['bs1']=='B']
    MTF_targetBuy['UpDown'] = MTF_targetBuy['price1']>MTF_targetBuy['price1'].shift()
    MTF_targetBuy['Fluidity'] = 0
    MTF_targetBuy['HFT_BuyDemand'] = MTF_targetBuy['UpDown']*MTF_targetBuy['HFT']*MTF_targetBuy['vol1']
    HFT_BuyDemand = (MTF_targetBuy.groupby(['date','Time_labels'])['HFT_BuyDemand'].sum()/MTF_targetBuy['vol1'].sum()).reset_index()

    MTF_targetSell['Supply'] = MTF_targetSell['price1']>MTF_targetSell['price1'].shift()
    MTF_targetSell['HFT_SellSupply'] = MTF_targetSell['Supply']*MTF_targetSell['HFT']*MTF_targetSell['vol1']
    HFT_SellSupply = (MTF_targetSell.groupby(['date','Time_labels'])['HFT_SellSupply'].sum()/MTF_targetSell['vol1'].sum()).reset_index()

    MTF_targetBuy['Supply'] = MTF_targetBuy['price1']<MTF_targetBuy['price1'].shift()
    MTF_targetBuy['HFT_BuySupply'] = MTF_targetBuy['Supply']*MTF_targetBuy['HFT']*MTF_targetBuy['vol1']
    HFT_BuySupply = (MTF_targetBuy.groupby(['date','Time_labels'])['HFT_BuySupply'].sum()/MTF_targetBuy['vol1'].sum()).reset_index()

    All1 = HFT_SellDemand.join(HFT_BuyDemand['HFT_BuyDemand'])
    All1 = All1.join(HFT_JoinRate['HFT_JoinRate'])
    All1['HFTDemand'] = All1['HFT_BuyDemand']+All1['HFT_SellDemand']

    All2 = HFT_SellSupply.join(HFT_BuySupply['HFT_BuySupply'])
    All2['HFTSupply'] = All2['HFT_BuySupply']+All2['HFT_SellSupply']
    All = All1.join(All2['HFTSupply']).drop(columns=['HFT_BuyDemand','HFT_SellDemand'])
    return HFT_JoinRate, All, MTF_target2

In [142]:
# %%time
#切割MTF檔計算HFT參與率
# 切割所屬樣本
#先創一段時間樣本讓並記錄OSF委託所屬樣本區間
def JoinRate1M():
    Time_list = np.arange(845,1347,1)
    Time_list = Time_list[Time_list%100 < 60]
    Time_labels = np.delete(Time_list,0).astype(str)
    MTF_target2 = MTF_target.copy()
    MTF_target2['time'] = MTF_target2['daytime'].apply(lambda x: (x.hour)*100+x.minute)
    MTF_target2['Time_labels'] = pd.cut(MTF_target2['time'], Time_list, right = False, labels = Time_labels)

    #貼標記給所有HFT後計算參與率
    MTF_target2['HFT'] = 0
    for i in range(len(Record_HFT)):
        MTF_target2.loc[(Record_HFT.loc[i,'date']==MTF_target2['date'])&(Record_HFT.loc[i,'id']==MTF_target2['id']),'HFT'] = 1
    MTF_target2['HFTVol'] = MTF_target2['HFT']*MTF_target2['vol1']
    HFT_JoinRate = (MTF_target2.groupby(['date','Time_labels'])['HFTVol'].sum()/MTF_target2.groupby(['date','Time_labels'])['vol1'].sum())
    HFT_JoinRate = HFT_JoinRate.reset_index().rename(columns={0:'HFT_JoinRate'})

    #計算HFT流動性供需
    MTF_targetSell = MTF_target2[MTF_target2['bs1']=='S']
    MTF_targetSell['UpDown'] = MTF_targetSell['price1']<MTF_targetSell['price1'].shift()
    MTF_targetSell['Fluidity'] = 0
    MTF_targetSell['HFT_SellDemand'] = MTF_targetSell['UpDown']*MTF_targetSell['HFT']*MTF_targetSell['vol1']
    HFT_SellDemand = (MTF_targetSell.groupby(['date','Time_labels'])['HFT_SellDemand'].sum()/MTF_targetSell['vol1'].sum()).reset_index()

    MTF_targetBuy = MTF_target2[MTF_target2['bs1']=='B']
    MTF_targetBuy['UpDown'] = MTF_targetBuy['price1']>MTF_targetBuy['price1'].shift()
    MTF_targetBuy['Fluidity'] = 0
    MTF_targetBuy['HFT_BuyDemand'] = MTF_targetBuy['UpDown']*MTF_targetBuy['HFT']*MTF_targetBuy['vol1']
    HFT_BuyDemand = (MTF_targetBuy.groupby(['date','Time_labels'])['HFT_BuyDemand'].sum()/MTF_targetBuy['vol1'].sum()).reset_index()

    MTF_targetSell['Supply'] = MTF_targetSell['price1']>MTF_targetSell['price1'].shift()
    MTF_targetSell['HFT_SellSupply'] = MTF_targetSell['Supply']*MTF_targetSell['HFT']*MTF_targetSell['vol1']
    HFT_SellSupply = (MTF_targetSell.groupby(['date','Time_labels'])['HFT_SellSupply'].sum()/MTF_targetSell['vol1'].sum()).reset_index()

    MTF_targetBuy['Supply'] = MTF_targetBuy['price1']<MTF_targetBuy['price1'].shift()
    MTF_targetBuy['HFT_BuySupply'] = MTF_targetBuy['Supply']*MTF_targetBuy['HFT']*MTF_targetBuy['vol1']
    HFT_BuySupply = (MTF_targetBuy.groupby(['date','Time_labels'])['HFT_BuySupply'].sum()/MTF_targetBuy['vol1'].sum()).reset_index()

    All1 = HFT_SellDemand.join(HFT_BuyDemand['HFT_BuyDemand'])
    All1 = All1.join(HFT_JoinRate['HFT_JoinRate'])
    All1['HFTDemand'] = All1['HFT_BuyDemand']+All1['HFT_SellDemand']

    All2 = HFT_SellSupply.join(HFT_BuySupply['HFT_BuySupply'])
    All2['HFTSupply'] = All2['HFT_BuySupply']+All2['HFT_SellSupply']
    All = All1.join(All2['HFTSupply']).drop(columns=['HFT_BuyDemand','HFT_SellDemand'])
    return HFT_JoinRate, All, MTF_target2

In [130]:
# %%time
#NearDepth
def NearDepth5M():
    #NearDepth
    Time_list = [845, 850, 855, 900, 905 ,910, 915, 920, 925, 930, 935, 940, 945, 950, 955, 1000, 1005, 1010, 1015, 1020, 1025, 1030, 1035,
                1040, 1045, 1050, 1055, 1100, 1105, 1110, 1115, 1120, 1125, 1130, 1135, 1140, 1145, 1150, 1155, 1200, 1205, 1210,1215,
                1220, 1225, 1230, 1235, 1240, 1245, 1250, 1255, 1300, 1305, 1310, 1315, 1320, 1325, 1330, 1335, 1340, 1346]
    Time_labels =['850', '855', "900", "905" ,"910", "915", "920", "925", "930", "935", "940", "945", "950", "955", "1000", "1005", "1010",
                  "1015", "1020", "1025", "1030", "1035", "1040", "1045", "1050", "1055", "1100", "1105", "1110", '1115', '1120', '1125', 
                  '1130', '1135', '1140', '1145', '1150', '1155', '1200', '1205', '1210', '1215', '1220', '1225', '1230', '1235', '1240',
                  '1245', '1250', '1255', '1300', '1305', '1310', '1315', '1320', '1325', '1330', '1335', '1340', '1346']
    DSP_target2 = DSP_target

    DSP_target2['NearDepth'] = (DSP_target2['bidv1']+DSP_target2['bidv2']+DSP_target2['bidv3']+DSP_target2['bidv4']+DSP_target2['bidv5']
                                +DSP_target2['askv1']+DSP_target2['askv2']+DSP_target2['askv3']+DSP_target2['askv4']+DSP_target2['askv5'])
    DSP_target2['time']= DSP_target2['daytime'].apply(lambda x: (x.hour)*100+x.minute)
    DSP_target2['Time_labels'] = pd.cut(DSP_target2['time'], Time_list, right = False, labels = Time_labels)

    DSP_target3 = DSP_target2[['date','daytime','NearDepth','Time_labels']]
    DSP_target3 = DSP_target3.groupby(['date','daytime','Time_labels'])['NearDepth'].mean().reset_index() #因為每個row都是一秒所以直接平均就可以了
    NearDepth = DSP_target3[['date','Time_labels','NearDepth']].groupby(['date','Time_labels'])['NearDepth'].sum().reset_index()

    #計算市場品質HighLow
    DSP_target4 = DSP_target2[['date','daytime','bid1','bidv1','ask1','askv1','time','Time_labels']]

    DSP_target4['MidPoint'] = (DSP_target4['bid1']+DSP_target4['ask1'])/2
    MidPointRange = (DSP_target4.groupby(['date','Time_labels'])['MidPoint'].max()-DSP_target4.groupby(['date','Time_labels'])['MidPoint'].min()
                    ).reset_index().rename(columns={'MidPoint':'MidPointRange'})
    MidPointMean = DSP_target4.groupby(['date','daytime','Time_labels'])['MidPoint'].mean().reset_index().drop(['daytime'],axis= 1)          #每個row都是一秒所以直接平均
    MidPointHighLow = pd.merge(MidPointRange,MidPointMean,on=['date','Time_labels'])
    MidPointHighLow['MidPointHighLow'] = MidPointHighLow['MidPointRange']/MidPointHighLow['MidPoint']
    MidPointHighLow = MidPointHighLow.groupby(['date','Time_labels'])['MidPointHighLow'].max().reset_index()

    DSP_target4['VolMidPoint'] = (DSP_target4['bid1']*DSP_target4['bidv1']+DSP_target4['ask1']*DSP_target4['askv1'])/(DSP_target4['bidv1']+DSP_target4['askv1'])
    VolMidPointRange = (DSP_target4.groupby(['date','Time_labels'])['VolMidPoint'].max()-DSP_target4.groupby(['date','Time_labels'])['VolMidPoint'].min()
                             ).reset_index().rename(columns={'VolMidPoint':'VolMidPointRange'})
    VolweightMidPointMean = DSP_target4.groupby(['date','daytime','Time_labels'])['VolMidPoint'].mean().reset_index().drop(['daytime'],axis= 1)
    VolMidPointHighLow = pd.merge(VolMidPointRange,VolweightMidPointMean,on=['date','Time_labels'])
    VolMidPointHighLow['VolMidPointHighLow'] = VolMidPointHighLow['VolMidPointRange']/VolMidPointHighLow['VolMidPoint']
    VolMidPointHighLow = VolMidPointHighLow.groupby(['date','Time_labels'])['VolMidPointHighLow'].max().reset_index()

    #計算市場品質Volatility
    RollingPeriod = 60
    Volatility = MTF_target2[['date','daytime','price1','vol1' ,'Time_labels']].sort_values(by=['daytime'])
    Volatility['VolatilityN'] = Volatility['price1'].rolling(RollingPeriod).std()
    VolatilityN = Volatility.groupby(['date','Time_labels']).mean().reset_index().drop(['price1','vol1'],axis= 1)

    #時間加權Volatility微調時間經過
    modify = Volatility.groupby(['date'])['price1'].last().reset_index()
    modify['date']+=datetime.timedelta(days=1)
    modify['time'] = datetime.time(8,45)
    modify['vol1'] = 0
    Volatility1 = Volatility.groupby(['date','daytime','price1',"Time_labels"])['vol1'].sum().reset_index()
    Volatility1['time'] = Volatility1['daytime'].apply(lambda x:x.time())
    Volatility1 = Volatility1.drop(['daytime'],axis= 1)
    Volatility1 = pd.concat([Volatility1,modify],axis=0).sort_values(by=['date','time'])
    Volatility1["time"] = Volatility1["time"].apply(lambda x:datetime.datetime.combine(datetime.datetime.min,x)-datetime.datetime.min)
    Volatility1['timelag'] = Volatility1.groupby(['date'])['time'].shift() 
    Volatility1['timedelta'] = Volatility1['time']-Volatility1['timelag']
    Volatility1['VolatilityN']= Volatility1['price1'].rolling(RollingPeriod,min_periods=1).std()
    Volatility1.loc[:,'VolatilityT'] = Volatility1.loc[:,'VolatilityN']*Volatility1.loc[:,'timedelta'].apply(lambda x:x.microseconds)/300000
    VolatilityT = (Volatility1.groupby(['date','Time_labels'])['VolatilityT'].mean()).reset_index()
    VolatilityT['Time_labels'] = VolatilityT['Time_labels'].apply(lambda x:int(x))
    VolatilityT = VolatilityT.sort_values(by=['date','Time_labels']).reset_index(drop=True)
    VolatilityT['Time_labels'] = VolatilityT['Time_labels'].apply(lambda x:str(x))
    return VolatilityT, VolatilityN, NearDepth, MidPointHighLow, VolMidPointHighLow

In [141]:
#NearDepth
def NearDepth1M():
    Time_list = np.arange(845,1347,1)
    Time_list = Time_list[Time_list%100 < 60]
    Time_labels = np.delete(Time_list,0).astype(str)
    DSP_target2 = DSP_target

    DSP_target2['NearDepth'] = (DSP_target2['bidv1']+DSP_target2['bidv2']+DSP_target2['bidv3']+DSP_target2['bidv4']+DSP_target2['bidv5']
                                +DSP_target2['askv1']+DSP_target2['askv2']+DSP_target2['askv3']+DSP_target2['askv4']+DSP_target2['askv5'])
    DSP_target2['time']= DSP_target2['daytime'].apply(lambda x: (x.hour)*100+x.minute)
    DSP_target2['Time_labels'] = pd.cut(DSP_target2['time'], Time_list, right = False, labels = Time_labels)

    DSP_target3 = DSP_target2[['date','daytime','NearDepth','Time_labels']]
    DSP_target3 = DSP_target3.groupby(['date','daytime','Time_labels'])['NearDepth'].mean().reset_index() #因為每個row都是一秒所以直接平均就可以了
    NearDepth = DSP_target3[['date','Time_labels','NearDepth']].groupby(['date','Time_labels'])['NearDepth'].sum().reset_index()

    #計算市場品質HighLow
    DSP_target4 = DSP_target2[['date','daytime','bid1','bidv1','ask1','askv1','time','Time_labels']]

    DSP_target4['MidPoint'] = (DSP_target4['bid1']+DSP_target4['ask1'])/2
    MidPointRange = (DSP_target4.groupby(['date','Time_labels'])['MidPoint'].max()-DSP_target4.groupby(['date','Time_labels'])['MidPoint'].min()
                    ).reset_index().rename(columns={'MidPoint':'MidPointRange'})
    MidPointMean = DSP_target4.groupby(['date','daytime','Time_labels'])['MidPoint'].mean().reset_index().drop(['daytime'],axis= 1)          #每個row都是一秒所以直接平均
    MidPointHighLow = pd.merge(MidPointRange,MidPointMean,on=['date','Time_labels'])
    MidPointHighLow['MidPointHighLow'] = MidPointHighLow['MidPointRange']/MidPointHighLow['MidPoint']
    MidPointHighLow = MidPointHighLow.groupby(['date','Time_labels'])['MidPointHighLow'].max().reset_index()

    DSP_target4['VolMidPoint'] = (DSP_target4['bid1']*DSP_target4['bidv1']+DSP_target4['ask1']*DSP_target4['askv1'])/(DSP_target4['bidv1']+DSP_target4['askv1'])
    VolMidPointRange = (DSP_target4.groupby(['date','Time_labels'])['VolMidPoint'].max()-DSP_target4.groupby(['date','Time_labels'])['VolMidPoint'].min()
                             ).reset_index().rename(columns={'VolMidPoint':'VolMidPointRange'})
    VolweightMidPointMean = DSP_target4.groupby(['date','daytime','Time_labels'])['VolMidPoint'].mean().reset_index().drop(['daytime'],axis= 1)
    VolMidPointHighLow = pd.merge(VolMidPointRange,VolweightMidPointMean,on=['date','Time_labels'])
    VolMidPointHighLow['VolMidPointHighLow'] = VolMidPointHighLow['VolMidPointRange']/VolMidPointHighLow['VolMidPoint']
    VolMidPointHighLow = VolMidPointHighLow.groupby(['date','Time_labels'])['VolMidPointHighLow'].max().reset_index()

    #計算市場品質Volatility
    RollingPeriod = 60
    Volatility = MTF_target2[['date','daytime','price1','vol1' ,'Time_labels']].sort_values(by=['daytime'])
    Volatility['VolatilityN'] = Volatility['price1'].rolling(RollingPeriod).std()
    VolatilityN = Volatility.groupby(['date','Time_labels']).mean().reset_index().drop(['price1','vol1'],axis= 1)

    #時間加權Volatility微調時間經過
    modify = Volatility.groupby(['date'])['price1'].last().reset_index()
    modify['date']+=datetime.timedelta(days=1)
    modify['time'] = datetime.time(8,45)
    modify['vol1'] = 0
    Volatility1 = Volatility.groupby(['date','daytime','price1',"Time_labels"])['vol1'].sum().reset_index()
    Volatility1['time'] = Volatility1['daytime'].apply(lambda x:x.time())
    Volatility1 = Volatility1.drop(['daytime'],axis= 1)
    Volatility1 = pd.concat([Volatility1,modify],axis=0).sort_values(by=['date','time'])
    Volatility1["time"] = Volatility1["time"].apply(lambda x:datetime.datetime.combine(datetime.datetime.min,x)-datetime.datetime.min)
    Volatility1['timelag'] = Volatility1.groupby(['date'])['time'].shift() 
    Volatility1['timedelta'] = Volatility1['time']-Volatility1['timelag']
    Volatility1['VolatilityN']= Volatility1['price1'].rolling(RollingPeriod,min_periods=1).std()
    Volatility1.loc[:,'VolatilityT'] = Volatility1.loc[:,'VolatilityN']*Volatility1.loc[:,'timedelta'].apply(lambda x:x.microseconds)/300000
    VolatilityT = (Volatility1.groupby(['date','Time_labels'])['VolatilityT'].mean()).reset_index()
    VolatilityT['Time_labels'] = VolatilityT['Time_labels'].apply(lambda x:int(x))
    VolatilityT = VolatilityT.sort_values(by=['date','Time_labels']).reset_index(drop=True)
    VolatilityT['Time_labels'] = VolatilityT['Time_labels'].apply(lambda x:str(x))
    return VolatilityT, VolatilityN, NearDepth, MidPointHighLow, VolMidPointHighLow

In [150]:
def Total_Variable(name):
    #控制變數
    Control_Return = MTF_target2.groupby(['date','Time_labels'])['price1'].last().reset_index()
    Control_Return['return'] = Control_Return['price1'].pct_change()
    Control_Return = Control_Return.drop(['price1'],axis= 1)

    Control_Volume = MTF_target2.groupby(['date','Time_labels'])['vol1'].sum().reset_index().rename(columns={'vol1':'volume'})

    #匯集全部變數
    Ttotal_Variable = All.join(NearDepth['NearDepth'])
    Ttotal_Variable = Ttotal_Variable.join(MidPointHighLow['MidPointHighLow'])
    Ttotal_Variable = Ttotal_Variable.join(VolMidPointHighLow['VolMidPointHighLow'])
    Ttotal_Variable = Ttotal_Variable.join(VolatilityN['VolatilityN'])
    Ttotal_Variable = Ttotal_Variable.join(VolatilityT['VolatilityT'])
    Ttotal_Variable = Ttotal_Variable.join(Control_Return['return'])
    Ttotal_Variable = Ttotal_Variable.join(Control_Volume['volume'])
    Ttotal_Variable["return"] = Ttotal_Variable["return"].fillna(0)
    Ttotal_Variable.to_csv('Ttotal_Variable_'+ date +name+'.csv')
    return Ttotal_Variable

In [65]:
def reading(name):
    while True:
        for j in name:
            if j != "POI":
                Folder_Path = initial_place + "\\" + j
                os.chdir(Folder_Path)
                file_list = np.array(os.listdir())
                file_list = file_list[start_index:end_index]
                yield pd.read_csv(Folder_Path + '\\'+ file_list[i],sep=',',encoding='big5')
            elif j == "POI":
                yield j

## 併發holding period

In [78]:
def holding_period():
    POI_target1 = POI_target[['date','id','daytime','contract','ybuyre','ysellre']]
    POI_target1['daytime'] +=datetime.timedelta(hours=8, minutes=45)
    POI_target1['oc'] = 0
    POI_target1['vol'] = POI_target1['ybuyre']+POI_target1['ysellre']     #不論昨日多空各留多少只要沒看到平倉就要計算持倉時間
    POI_target1 = POI_target1.drop(['ybuyre','ysellre'],axis= 1)

    #增加最後一行方便計算持倉到收盤的口數
    POI_target2 = POI_target[['date','id','daytime','contract','tbuyre','tsellre']]
    POI_target2['daytime'] += datetime.timedelta(hours=13,minutes=44, seconds=59, milliseconds =999)
    POI_target2['oc'] = 3
    POI_target2['vol'] = POI_target2['tbuyre']+POI_target2['tsellre']     #不論今日多空各留多少只要沒看到平倉就要計算持倉時間
    POI_target2 = POI_target2.drop(['tbuyre','tsellre'],axis= 1)

    #處理成交檔再concat計算持倉時間
    MTF_target1 = MTF_target[['date','id','daytime','contract1','oc','vol1']].rename(columns={'vol1':'vol','contract1':'contract'})
    Calculate_df_cols = ['date','daytime','contract','id','oc','vol','timedelta','OpenInterest','time_lag','InterestChange','CumulativeTime','TotalOpen']
    Calculate_temp = pd.DataFrame(columns=Calculate_df_cols)
    Calculate_df = pd.concat([Calculate_temp,POI_target1,POI_target2,MTF_target1],axis=0).sort_values(['daytime']).reset_index(drop=True)
    Calculate_df = Calculate_df.ix[:,Calculate_df_cols]
    Calculate_df2 = pd.DataFrame(columns = Calculate_df_cols)

    def check_yield():
        while True: 
            b = yield
            if b<0:
                Calculate_df1[a,5] += abs(b)
                Calculate_df1[:,9] = Calculate_df1[:,5]*(((Calculate_df1[:,4]==0)|(Calculate_df1[:,4]==2))*2-1)
    Datelist = pd.unique(POI_target['date'])
    for i in Datelist:
        if i != date_now:
            continue
        Calculate_df1 = Calculate_df[Calculate_df['date']==i]
        Calculate_df1.loc[:,'InterestChange'] = (Calculate_df1['vol']*Calculate_df1['oc'].apply(lambda x:((x==0)or(x==2))*2-1)).astype(float)
        Calculate_df1.loc[:,'time_lag'] = Calculate_df1.groupby(['id'])['daytime'].shift(1)
        Calculate_df1.loc[:,'OpenInterest'] = Calculate_df1.groupby(['date','id'])['InterestChange'].cumsum()-Calculate_df1['InterestChange']
        Calculate_df1 = Calculate_df1.values
        Calculate_df1[:,6] = (Calculate_df1[:,1] - Calculate_df1[:,8])
        Idlist = pd.unique(Calculate_df1[:,3])
        c = check_yield()
        next(c)
        for j in Idlist:
            a = np.where(Calculate_df1[:,3]==j)[0][0]
            b = np.sort(Calculate_df1[Calculate_df1[:,3]==j][:,7])[0]
            c.send(b)
        Calculate_df1 = pd.DataFrame(Calculate_df1,columns = Calculate_df_cols)
        Calculate_df1.loc[:,'InterestChange'] = Calculate_df1['InterestChange'].astype(float)
        Calculate_df1.loc[:,'OpenInterest'] = Calculate_df1.groupby(['id'])['InterestChange'].cumsum()-Calculate_df1['InterestChange']
        Calculate_df1.loc[:,'CumulativeTime'] = Calculate_df1['timedelta']*Calculate_df1['OpenInterest']
        Calculate_df1.loc[:,'TotalOpen'] = Calculate_df1['vol']*Calculate_df1['oc'].apply(lambda x:((x==0)or(x==2)))
        Calculate_df1 = Calculate_df1.fillna(0)
        Calculate_df2 = Calculate_df2.append(Calculate_df1)

    HoldingPeriod = pd.merge(Calculate_df2.groupby(['date','id'])['CumulativeTime'].sum().reset_index(),Calculate_df2.groupby(['date','id'])['TotalOpen'].sum().reset_index(), on=['date','id'], how='outer')
    HoldingPeriod['TotalOpen'] = HoldingPeriod['TotalOpen'].astype(float)
    HoldingPeriod['HoldingPeriod'] = (HoldingPeriod['CumulativeTime']/HoldingPeriod['TotalOpen'])
    HoldingPeriod = HoldingPeriod.dropna()
    return HoldingPeriod

In [145]:
start = time.time()
print("請Total檔案放置與4個檔案資料夾(DSP,MTF,OSF,POI])同一階層接收資料")
       #要拼接的文件夹及其完整路径，注意不要包含中文
os.chdir(initial_place + "\\" + "poi")
poi_list = np.array(os.listdir())
poi_list = poi_list[np.where(poi_list == poi_file_start)[0][0]:np.where(poi_list == poi_file_end)[0][0]+1]
# poi_start = np.where(poi_list == poi_file_start)[0][0]
# poi_end =  np.where(poi_list == poi_file_end)[0][0]+1

os.chdir(initial_place + "\\" + "OSF")
start_index = np.where(file_list == start_file)[0][0]
end_index = np.where(file_list == end_file)[0][0]+1
file_list = file_list[start_index:end_index]

for i in range(len(file_list)):
    date_now = ""
    os.chdir(initial_place + "\\" + "OSF")
#     start_index = np.where(file_list == start_file)[0][0]
#     end_index = np.where(file_list == end_file)[0][0]+1
    file_list = os.listdir()
    file_list = file_list[start_index:end_index]
    try:
        transfer_list = [file_list[i][-12:-4],file_list[i+1][-12:-4],file_list[i+2][-12:-4]]
    except:
        try:
            transfer_list = [file_list[i][-12:-4],file_list[i+1][-12:-4]]
        except:
            transfer_list = [file_list[i][-12:-4]]
    year_month_now = file_list[i][-12:-6]
    date = file_list[i][-12:-4]
    date_now = datetime.datetime.strptime(date,"%Y%m%d").date()
    #將所有文件名存入一个列表
    read = reading(name)

    OSF_RawData = next(read)
    MTF_RawData = next(read)
    DSP_RawData = next(read)

    POI = next(read)
    Folder_Path = initial_place + "\\" + POI
    if poi_true == True:
        POI_RawData = pd.read_csv(Folder_Path + '\\'+ poi_list[poi_count],sep=',',encoding='big5')

        poi_true =False
    if year_month_now not in poi_list[poi_count]:
        poi_count += 1
        POI_RawData = pd.read_csv(Folder_Path + '\\'+ poi_list[poi_count],sep=',',encoding='big5')
    year_month_now = file_list[i][-12:-6]
    date = file_list[i][-12:-4]
    date_now = datetime.datetime.strptime(date,"%Y%m%d").date()
    OSF_RawData['daytime'] = pd.to_datetime(OSF_RawData['daytime'] , format='%Y-%m-%d %H:%M:%S')
    MTF_RawData['daytime'] = pd.to_datetime(MTF_RawData['daytime'] , format='%Y-%m-%d %H:%M:%S')
    DSP_RawData['daytime'] = pd.to_datetime(DSP_RawData['daytime'] , format='%Y-%m-%d %H:%M:%S')
    POI_RawData['daytime'] = pd.to_datetime(POI_RawData['daytime'].astype(str), format='%Y-%m-%d')
    OSF_TXF, MTF_TXF, DSP_TXF, POI_TXF = Data_Pick()
    OSF_target, POI_target, MTF_target, DSP_target = Transfer()
    HoldingPeriod = holding_period()
    Record_HFT,RankData = Vol_Calculate()
    HFT_JoinRate, All, MTF_target2 = JoinRate5M()
    VolatilityT, VolatilityN, NearDepth, MidPointHighLow, VolMidPointHighLow = NearDepth5M()
    SaveFile_Path =  initial_place
    os.chdir(SaveFile_Path  + "Features_record")
    Features_record = Features_Record("")
    os.chdir(SaveFile_Path  + "Ttotal_Variable")
    Ttotal_Variable = Total_Variable("_5m")
    gc.collect()
    HFT_JoinRate, All, MTF_target2 = JoinRate1M()
    VolatilityT, VolatilityN, NearDepth, MidPointHighLow, VolMidPointHighLow = NearDepth1M()
    # 位置沒改變不須切換
    Ttotal_Variable = Total_Variable("_1m")

    
print(time.time()-start)


請Total檔案放置與4個檔案資料夾(DSP,MTF,OSF,POI])同一階層接收資料


C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = v

C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\user\Anaconda3\lib\site-packages\ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-

143.62701511383057


# 合併檔案

In [68]:
def read_merge(j):
    while True:
        yield pd.read_csv(file_list[j+1],encoding='big5')

## 併feature只要改位置跟最後儲存名稱即可

In [69]:
os.chdir(initial_place + "\\" + "Ttotal_Variable")
file_list = np.array(os.listdir())
Total_merge = pd.read_csv(file_list[0])
for j in range(len(file_list)-1):
    RM = read_merge(j)
    temp = next(RM)
    Total_merge = Total_merge.append(temp)
    RM.send(j)
Total_merge = Total_merge.reset_index(drop = True)
Total_merge.to_csv("Total_data.csv")

In [70]:
os.chdir(initial_place + "\\" + "Features_record")
file_list = np.array(os.listdir())
# 需要使用變數記得改這邊的變數名稱
Total_merge = pd.read_csv(file_list[0])
for j in range(len(file_list)-1):
    RM = read_merge(j)
    Total_merge = (Total_merge + next(RM))/len(file_list)
    RM.send(j)
Total_merge = Total_merge.reset_index(drop = True)
Total_merge.to_csv("Features_record.csv")